In [1]:
import matplotlib.pyplot as plt
import numpy as np
#import math as mt
from mpmath import *
from scipy.optimize import curve_fit, minimize
import scipy.special as sp_spec

# Fit of Transfer Function

In [2]:
gizi=0.04 
giz=0.01
Eiz=-65.
Eizi=-60.
Ee=0.0
Ei=-80.0
Qi=5.0
Qe=1.5
tauize=1
tauizi=1
tau_adap=1
a_adap=1
d_adap=15
b_adap=0
tause=5e-3
tausi=5e-3
Ke=8000*0.05
Ki=2000*0.05
ue=0

def sv(ne,ni):
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    
    ui=0
    #ue=0
    ue=0.5*d_adap*ne*tau_adap/a_adap    #   du/dt=(a*(b*v-u)) + nu*d /Tu
   
    muve=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    muvi=((2*gizi*Eizi+muge+mugi)-np.sqrt((2*gizi*Eizi+muge+mugi)**2-4*gizi*(gizi*Eizi**2+muge*Ee+mugi*Ei-ui)))/(2*gizi)
   
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
 
    be=Qe*(Ee-muve)
    bi=Qi*(Ei-muvi)
    ce=tause
    ci=tausi
    
    sv=np.sqrt(Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2)))
    
    #print('sv=',sv)
    return sv
    
def tauv(ne,ni):
    
    ne=ne+1e-9
    ni=ni+1e-9
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    #mug=muge+mugi+giz
    ui=0
    #ue=0
    ue=0.5*d_adap*ne**tau_adap/a_adap
    muve=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    muvi=((2*gizi*Eizi+muge+mugi)-np.sqrt((2*gizi*Eizi+muge+mugi)**2-4*gizi*(gizi*Eizi**2+muge*Ee+mugi*Ei-ui)))/(2*gizi)

    
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
    be=Qe*(Ee-muve)
    bi=Qi*(Ei-muvi)
    ce=tause
    ci=tausi
    
    svv2=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2))
    
    tauv=0.5*(Ke*ne*(be**2*ce**4/(2*np.pi*tauize**2))+Ki*ni*(bi**2*ci**4/(2*np.pi*tauizi**2)))/(svv2+1e-9)
    
    #print('tauv=',tauv)
    return tauv

In [3]:

def TF(nu,Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,giz,Eiz,d_ad):

    ne,ni=nu
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    mug=muge+mugi+giz
    ui=0

    ue=0.5*d_adap*ne*tau_adap/a_adap  #u_st=(b*nu+a*b*v)*tau/a#du/dt=(a*(b*v-u))/Tu
    
    muvo=-45
    dmuvo=2
    svo=4
    dsvo=5
    tauvo=0.005
    dtauvo=0.05
    
    muv=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    
    Pscale=1.
    noutf=Pscale*(sp_spec.erfc(((Po + Pmuv*(muv-muvo)/dmuvo + Psv*(sv(ne,ni)-svo)/dsvo + Ptauv*(tauv(ne,ni)-tauvo)/dtauvo \
            +Pvsv*(muv-muvo)*(sv(ne,ni)-svo)/(dsvo*dmuvo) + Pvtauv*(muv-muvo)*(tauv(ne,ni)-tauvo)/(dtauvo*dmuvo)\
            +Psvtauv*(sv(ne,ni)-svo)*(tauv(ne,ni)-tauvo)/(dtauvo*dsvo) + Pvv*(muv-muvo)**2/(dmuvo*dmuvo) \
            +Ptt*(tauv(ne,ni)-tauvo)**2/(dtauvo*dtauvo) + Pss*(sv(ne,ni)-svo)**2/(dsvo*dsvo))-muv)/(np.sqrt(2)*sv(ne,ni)) ))/(2*tauv(ne,ni))

        
    return noutf

In [4]:
def Tresh_func(nu,Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,giz,Eiz,d_adap):
    ne,ni=nu

    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    mug=muge+mugi+giz
    ui=0
   
    ue=0.5*d_adap*ne*tau_adap/a_adap #u_st=(b*nu+a*b*v)*tau/a#du/dt=(a*(b*v-u))/Tu

    muvo=-45
    dmuvo=2
    svo=4
    dsvo=5
    tauvo=0.005
    dtauvo=0.05

    muv=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)


    Vteff2=(Po + Pmuv*(muv-muvo)/dmuvo + Psv*(sv(ne,ni)-svo)/dsvo + Ptauv*(tauv(ne,ni)-tauvo)/dtauvo \
            +Pvsv*(muv-muvo)*(sv(ne,ni)-svo)/(dsvo*dmuvo) + Pvtauv*(muv-muvo)*(tauv(ne,ni)-tauvo)/(dtauvo*dmuvo)\
            +Psvtauv*(sv(ne,ni)-svo)*(tauv(ne,ni)-tauvo)/(dtauvo*dsvo) + Pvv*(muv-muvo)**2/(dmuvo*dmuvo) \
            +Ptt*(tauv(ne,ni)-tauvo)**2/(dtauvo*dtauvo) + Pss*(sv(ne,ni)-svo)**2/(dsvo*dsvo))
    
    return Vteff2

In [5]:
def effective_Vthre(ne,ni,Fout,giz,Eiz,d_adap):
    
    Y=Fout
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    mug=muge+mugi+giz
    ui=0
    
    ue=0.5*d_adap*ne*tau_adap/a_adap#u_st=(b*nu+a*b*v)*tau/a#du/dt=(a*(b*v-u))/Tu
    
    
    muv=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)

    
    Vthre_eff = muv+np.sqrt(2)*sv(ne,ni)*sp_spec.erfcinv(\
                    Y*2.*tauv(ne,ni)) # effective threshold
    return Vthre_eff


In [6]:

#P=np.ones(10)*1e-1#np.array([-2.19840587e+05, -8.63334589e+04,  1.07805996e+05,  1.06227624e+05,2.39650033e+04,  2.35507810e+04, -1.02726656e+03, -8.33094033e+03,5.19218481e+03,  3.81483637e+01])#np.ones(10)*1e-3
#Fout=noutexpe2

def Res(P):
    
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
   # print(P)
    
    Vthre_eff=effective_Vthre(nue2,nui2,Fout,giz,Eiz,d_adap)
    vthre=Tresh_func((nue2,nui2),Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,giz,Eiz,d_adap)
    
    res=np.mean((Vthre_eff-vthre)**2)
    
    return res

In [7]:
def Res2(P):
    
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
    #print(P)
    
    res=np.mean((TF((nue2,nui2),Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,giz,Eiz,d_adap) - Fout)**2)
    
    return res

In [8]:

def Resin(P):
    
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
   # print(P)
    
    Vthre_eff=effective_Vthre(nue2,nui2,Fout,gizi,Eizi,0)
    vthre=Tresh_func((nue2,nui2),Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,gizi,Eizi,0)
    
    res=np.mean((Vthre_eff-vthre)**2)
    
    return res

def Res2in(P):
    
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
    #print(P)
    
    res=np.mean((TF((nue2,nui2),Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss,gizi,Eizi,0) - Fout)**2)
    
    return res

## Fit for the Excitatory neuron

In [9]:
############################### Fit for the excitatory neuron

# Define the nue, nui arrays according to the single neuron simulation (pairs of values used)

Npts=1  #Resolution of the measured TF
Npts2=1
nue=np.arange(0.0000002,100, Npts)
nui=np.arange(0.0000002,100, Npts2)
#noute=np.arange(0.1,201, Npts)

dim=len(nue)*len(nui)

    
nue2=np.ones(dim)        
#nui2=np.ones(dim)
counter=0        
for i in nue:
    for j in range(0,len(nui)):  
        nue2[counter]=i
        
        counter=counter+1
        
nui2=[]        
for j in range(0,len(nue)):
    nui2=np.concatenate((nui2, nui), axis=None)

# Load the simulations from the single neuron
noutexpe=np.load('ExpTF_exc_Nstp-100_Nstp2-100_new-params_0-100.npy')#('ExpTF_exc_Nstp100_new_noad_0-200.npy')

# Re-write the output of the simulation as an array for the pairs (nue,nui)
noutexpe2=np.ones(dim)

counter=0

for j in range(0,len(nue)):
    for i in range(0,len(nui)):
        noutexpe2[counter]=noutexpe[j,i]
        counter=counter+1


#noutexpe

In [10]:
# Remove all the problematic points (nans, etc.) to make the fit

svout=sv(nue2,nui2)
tvout=tauv(nue2,nui2)

#svout=sv(nue2,nui2)
#tvout=tauv(nue2,nui2)

jjindex=np.where(np.isnan(svout))


nue2=np.delete(nue2,jjindex)
nui2=np.delete(nui2,jjindex)
noutexpe2=np.delete(noutexpe2,jjindex)

tvout=tauv(nue2,nui2)

jjindex=np.where(np.isnan(tvout))

nue2=np.delete(nue2,jjindex)
nui2=np.delete(nui2,jjindex)
noutexpe2=np.delete(noutexpe2,jjindex)


jjindex=np.where(noutexpe2>0.)

nue2=nue2[jjindex]#,jjindex)
nui2=nui2[jjindex]
noutexpe2=noutexpe2[jjindex]

ijn=np.where(noutexpe2>40)
nue2=np.delete(nue2,ijn)
nui2=np.delete(nui2,ijn)
noutexpe2=np.delete(noutexpe2,ijn)



Fout=noutexpe2
Vthre_eff=effective_Vthre(nue2,nui2,Fout,giz,Eiz,d_adap)
ijn=np.where(np.isinf(Vthre_eff))
nue2=np.delete(nue2,ijn)
nui2=np.delete(nui2,ijn)
noutexpe2=np.delete(noutexpe2,ijn)


/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in sqrt


In [11]:
# Fit on Veff_threshold


Fout=noutexpe2

P=np.ones(10)*3e-1

plsq = minimize(Res, P, method='SLSQP',options={'ftol': 1e-17, 'disp': True, 'maxiter':400})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.169744844603893
            Iterations: 93
            Function evaluations: 1206
            Gradient evaluations: 93


In [12]:
plsq

     fun: 0.169744844603893
     jac: array([-1.86264515e-09, -4.09781933e-08, -9.31322575e-09,  1.30385160e-08,
        2.42143869e-08, -3.72529030e-09, -3.72529030e-09,  3.35276127e-08,
       -2.23517418e-08,  1.30385160e-08])
 message: 'Optimization terminated successfully.'
    nfev: 1206
     nit: 93
    njev: 93
  status: 0
 success: True
       x: array([-3.76123987e+01,  1.45498484e+00,  1.55830485e+01, -1.16746811e+02,
       -1.81808500e+00,  3.12268886e+00, -1.12578955e+02, -2.12936089e-02,
       -3.57132195e+00,  1.95857949e+00])

In [13]:
Pn=plsq.x

In [14]:
res=Res2(Pn)
print(res)

19661.344805846784


In [15]:
# Fit on TF


plsq2 = minimize(Res2, Pn, method='nelder-mead', options={'xtol': 1e-5, 'disp': True, 'maxiter':10000})

Optimization terminated successfully.
         Current function value: 28.864331
         Iterations: 3132
         Function evaluations: 4472


In [16]:
plsq2

 final_simplex: (array([[-2.78488672e+01,  7.75186875e-02,  2.38588248e+01,
         9.19915928e+00, -2.87046427e+00, -1.07017463e+00,
         6.16695768e+00, -9.97505733e-03,  5.95040383e+00,
         1.14656771e+00],
       [-2.78488675e+01,  7.75187413e-02,  2.38588247e+01,
         9.19915211e+00, -2.87046427e+00, -1.07017433e+00,
         6.16695199e+00, -9.97505865e-03,  5.95040338e+00,
         1.14656778e+00],
       [-2.78488667e+01,  7.75186107e-02,  2.38588255e+01,
         9.19916147e+00, -2.87046439e+00, -1.07017462e+00,
         6.16696134e+00, -9.97505794e-03,  5.95040422e+00,
         1.14656773e+00],
       [-2.78488673e+01,  7.75187052e-02,  2.38588253e+01,
         9.19915461e+00, -2.87046438e+00, -1.07017446e+00,
         6.16695420e+00, -9.97506039e-03,  5.95040369e+00,
         1.14656776e+00],
       [-2.78488674e+01,  7.75187264e-02,  2.38588250e+01,
         9.19915510e+00, -2.87046430e+00, -1.07017443e+00,
         6.16695485e+00, -9.97505890e-03,  5.95040353

In [17]:
Pn2=plsq2.x

In [18]:
# Parameters to use for the transfer function

print(Pn2)

[-2.78488672e+01  7.75186875e-02  2.38588248e+01  9.19915928e+00
 -2.87046427e+00 -1.07017463e+00  6.16695768e+00 -9.97505733e-03
  5.95040383e+00  1.14656771e+00]


In [19]:
Name2='RS_fit.txt'
f1 = open(Name2, 'w')
for i in range (0,len(Pn2)):
    f1.write(str(Pn2[i]) + "\n")
    
f1.close()

## Fit for the Inhibitory neuron

In [20]:
############# Fit for the inhibitory neuron

Npts=1  #Resolution of the measured TF
Npts2=1
nue=np.arange(0.0000002,100, Npts)
nui=np.arange(0.0000002,100, Npts2)
#noute=np.arange(0.1,201, Npts)

dim=len(nue)*len(nui)

    
nue2=np.ones(dim)        
counter=0        
for i in nue:
    for j in range(0,len(nui)):  
        nue2[counter]=i
        counter=counter+1
        
nui2=nui        
for j in range(0,len(nue)-1):
    nui2=np.concatenate((nui2, nui), axis=None)

noutexpe=np.load('ExpTF_inh_Nstp-100_Nstp2-100_new-params_0-100.npy')

noutexpe2=np.ones(dim)
adapexpe2=np.zeros(dim)
counter=0

for j in range(0,len(nue)):
    for i in range(0,len(nui)):
        noutexpe2[counter]=noutexpe[j,i]
        #adapexpe2[counter]=adapexpe[j,i]
        counter=counter+1


#noutexpe

In [21]:
# Remove all the problematic points (nans, etc.) to make the fit


svout=sv(nue2,nui2)
tvout=tauv(nue2,nui2)


jjindex=np.where(np.isnan(svout))


nue2=np.delete(nue2,jjindex)
nui2=np.delete(nui2,jjindex)
noutexpe2=np.delete(noutexpe2,jjindex)

tvout=tauv(nue2,nui2)

jjindex=np.where(np.isnan(tvout))

nue2=np.delete(nue2,jjindex)
nui2=np.delete(nui2,jjindex)
noutexpe2=np.delete(noutexpe2,jjindex)


jjindex=np.where(noutexpe2>0.)

nue2=nue2[jjindex]
nui2=nui2[jjindex]
noutexpe2=noutexpe2[jjindex]


Fout=noutexpe2
Vthre_eff=effective_Vthre(nue2,nui2,Fout,gizi,Eizi,0)
ijn=np.where(np.isinf(Vthre_eff))
nue2=np.delete(nue2,ijn)
nui2=np.delete(nui2,ijn)
noutexpe2=np.delete(noutexpe2,ijn)
ijn=np.where(noutexpe2>60)
nue2=np.delete(nue2,ijn)
nui2=np.delete(nui2,ijn)
noutexpe2=np.delete(noutexpe2,ijn)

Fout=noutexpe2

/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/home/ftesler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in sqrt


In [22]:
# Fit on TF

plsq2 = minimize(Res2in, Pn, method='nelder-mead', options={'xtol': 1e-5, 'disp': True, 'maxiter':10000})

Optimization terminated successfully.
         Current function value: 50.581430
         Iterations: 3501
         Function evaluations: 4945


In [23]:
plsq2

 final_simplex: (array([[-3.00178126e+01,  1.62788045e-01,  2.17624223e+01,
        -3.69954918e+01, -2.68888514e+00,  3.26691139e-01,
        -4.37286464e+01, -2.62828987e-03, -1.24850434e+01,
         2.69766790e+00],
       [-3.00178135e+01,  1.62788130e-01,  2.17624205e+01,
        -3.69955007e+01, -2.68888503e+00,  3.26691478e-01,
        -4.37286556e+01, -2.62828885e-03, -1.24850470e+01,
         2.69766673e+00],
       [-3.00178132e+01,  1.62788119e-01,  2.17624218e+01,
        -3.69954982e+01, -2.68888510e+00,  3.26691469e-01,
        -4.37286506e+01, -2.62829058e-03, -1.24850434e+01,
         2.69766767e+00],
       [-3.00178129e+01,  1.62788056e-01,  2.17624216e+01,
        -3.69954951e+01, -2.68888513e+00,  3.26691299e-01,
        -4.37286499e+01, -2.62828929e-03, -1.24850456e+01,
         2.69766720e+00],
       [-3.00178126e+01,  1.62788061e-01,  2.17624227e+01,
        -3.69954898e+01, -2.68888508e+00,  3.26691256e-01,
        -4.37286409e+01, -2.62828898e-03, -1.24850409

In [24]:
# Parameters to use for the TF (inhibitory)

Pn2=plsq2.x

In [25]:
Name2='FS_fit.txt'
f1 = open(Name2, 'w')
for i in range (0,len(Pn2)):
    f1.write(str(Pn2[i]) + "\n")
    
f1.close()